In [15]:
from PIL import Image
from torch.utils.data.dataset import Dataset
from scipy.misc import imread
import numpy as np
import torch
import torch.optim as optim
from core import networks

import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import copy

In [16]:
#Hyperparameters for training
mini_batch_size = 128
lambda_ = 0.01

In [17]:
#dataload (MNIST)
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),])),
        batch_size=mini_batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=False, transform=transforms.Compose([transforms.ToTensor(),])
                       ),
        batch_size=mini_batch_size, shuffle=True)

In [18]:
#custom regularization

import torch.nn as nn
from core.networks import BayesianNetwork
def custom_regularization(saver_net, trainer_net,mini_batch_size, lambda_, loss=None):
    
    mean_reg = 0
    sigma_reg = 0
    
    #net1, net2에서 각 레이어에 있는 mean, sigma를 이용하여 regularization 구현

    #각 모델에 module 접근
    for saver, trainer in zip(saver_net.modules(),trainer_net.modules()):
        
        #만약 BayesianNetwork 이면
        if isinstance(saver,BayesianNetwork) and  isinstance(trainer,BayesianNetwork):
            
            #Network 내부의 layer에 순차적으로 접근
            for saver_layer, trainer_layer in zip(saver.layer_arr, trainer.layer_arr):
            
            # calculate mean regularization
            
                mean_reg += lambda_*(torch.div(trainer_layer.weight_mu, saver_layer.weight_sigma)-torch.div(trainer_layer.weight_mu, trainer_layer.weight_sigma)).norm(2)
                mean_reg = mean_reg/(mini_batch_size*2)
                
            # calculate sigma_reg regularization
            
                sigma_reg += torch.sum(torch.div(trainer_layer.weight_sigma, saver_layer.weight_sigma) - torch.log(torch.div(trainer_layer.weight_sigma, saver_layer.weight_sigma)))
                sigma_reg = sigma_reg/(mini_batch_size*2)
                
                print (mean_reg, sigma_reg) # regularization value 확인
                
    loss = loss + mean_reg + sigma_reg 

    return loss

In [19]:
def train(saver_net,trainer_net, optimizer, epoch, mini_batch_size, lambda_):
    trainer_net.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        trainer_net.zero_grad()
        #         loss = net.sample_elbo(data, target) #홍준 코딩
        loss = custom_regularization(saver_net, trainer_net, mini_batch_size, lambda_)
        loss.backward()
        optimizer.step()


In [20]:
# GPU 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Model Initialization
#Saver_Net : mu = 0, sigma = log(1+exp(1))
#trainer_ner : mu = [-5,5], sigma = log(1+exp([-5,+5]))
saver_net = networks.BayesianNetwork(init_type = 'zero').to(DEVICE)
trainer_net = networks.BayesianNetwork(init_type = 'random').to(DEVICE)

optimizer = optim.Adam(saver_net.parameters())
optimizer = optim.Adam(trainer_net.parameters())

for epoch in range(10):

    #0. trainet_net variance init
    
    trainer_net.variance_init() #trainer net의 variance크게 init
    trainer_net = trainer_net.to(DEVICE)
    
    #1. trainet_net training 하는데 regularization을 위해서 saver_net의 정보 이용
    
    train(saver_net, trainer_net, optimizer, epoch, mini_batch_size, lambda_)

    #2. 1 batch가 끝나면 saver_net에 trainet_net을 복사 (weight = mean, sigma)
    
    saver_net = copy.deepcopy(trainer_net)
    

tensor(2.0934, device='cuda:0', grad_fn=<DivBackward0>) tensor(2722.7454, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1.4884, device='cuda:0', grad_fn=<DivBackward0>) tensor(1394.8323, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.2388, device='cuda:0', grad_fn=<DivBackward0>) tensor(39.9875, device='cuda:0', grad_fn=<DivBackward0>)


TypeError: add(): argument 'other' (position 1) must be Tensor, not NoneType